In [ ]:
!pip install shap

     |████████████████████████████████| 358kB 17.7MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491625 sha256=517bef76f209831b54ffd14f899422e081469d9632a498eebb1372a71033eda7
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


In [ ]:
!pip install eli5

     |████████████████████████████████| 112kB 16.5MB/s 


In [ ]:
import eli5
import shap
import numpy as np
import pandas as pd
import statsmodels.api as sm
import xgboost
from tqdm.notebook import tqdm

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


##For empirical part:

In [ ]:
X = pd.read_csv("x_clean.csv", index_col=0)
X.index = range(len(X))

In [ ]:
cate = pd.read_csv("CATE average.csv", index_col=0)
cate.index = X.index

In [ ]:
progs = ["job search", "vocational", "computer", "language"]

In [ ]:
index = pd.MultiIndex.from_product([progs, ["OLS", "perm", "shap"]], names=["program", "metric"])

In [ ]:
results = pd.DataFrame(index=index, columns=X.columns)

In [ ]:
for prog in progs:
  estimates = cate[prog]
  ols = sm.OLS(estimates, X).fit()
  results.loc[(prog, "OLS")] = ols.params
  boost = xgboost.XGBRegressor(n_estimators=500).fit(X, estimates)
  perm = eli5.sklearn.PermutationImportance(boost, refit=False, random_state=0, n_iter=1)
  perm.fit(X, estimates)
  results.loc[(prog, "perm")] = perm.feature_importances_
  explainer = shap.TreeExplainer(boost)
  shap_values = explainer(X)
  results.loc[(prog, "shap")] = np.abs(shap_values.values).mean(axis=0)

[23:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:06:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:08:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:10:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
results

const         age  ... ue_spells_last_2yrs  unemp_rate
program    metric                       ...                                
job search OLS    -0.16995  0.00396015  ...           0.0350906   0.0511715
           perm          0   0.0224347  ...          0.00996114   0.0275955
           shap          0   0.0325906  ...           0.0228916   0.0385865
vocational OLS     4.52097  0.00429222  ...          -0.0395107   0.0166385
           perm          0   0.0255955  ...           0.0322573   0.0163932
           shap          0   0.0332449  ...           0.0338657   0.0284181
computer   OLS     3.43015   0.0108097  ...         -0.00195633   0.0932527
           perm          0   0.0296682  ...         0.000242089  0.00595351
           shap          0   0.0816974  ...          0.00580148   0.0354276
language   OLS      3.5221  -0.0195298  ...         -0.00118293  -0.0180773
           perm          0   0.0589874  ...         0.000123264  0.00187852
           shap          0    0.139632  ...          0.00275506   0.0173971

[12 rows x 42 columns]

In [ ]:
results.to_csv("Variable importances.csv")

In [ ]:
for prog in progs:
  for metric in ["perm", "shap"]:
    results.loc[(prog, metric)] = results.loc[(prog, metric)] / results.loc[(prog, metric)].max()

In [ ]:
table = results[["age", "female", "past_income", "employability", "emp_share_last_2yrs", "foreigner_c", "other_mother_tongue", "unemp_rate", "ue_cw_allocation4"]].round(3).T

In [ ]:
table.to_csv("Variable importances formatted.csv")

In [ ]:
print(table.round(2).to_latex())

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
program & \multicolumn{3}{l}{job search} & \multicolumn{3}{l}{vocational} & \multicolumn{3}{l}{computer} & \multicolumn{3}{l}{language} \\
metric &        OLS &  perm &  shap &        OLS &  perm &  shap &      OLS &  perm &  shap &      OLS &  perm &  shap \\
\midrule
age                 &       0.00 &  0.05 &  0.15 &       0.00 &  0.05 &  0.22 &     0.01 &  0.03 &  0.12 &    -0.02 &  0.14 &  0.30 \\
female              &       0.03 &  0.00 &  0.03 &      -0.02 &  0.01 &  0.09 &     1.33 &  1.00 &  1.00 &    -0.67 &  0.52 &  0.70 \\
past\_income         &      -0.08 &  0.77 &  0.68 &      -0.07 &  0.68 &  0.92 &    -0.08 &  0.12 &  0.22 &     0.04 &  0.05 &  0.14 \\
employability       &      -0.03 &  0.01 &  0.04 &      -0.41 &  1.00 &  1.00 &    -0.34 &  0.05 &  0.13 &    -0.00 &  0.00 &  0.01 \\
emp\_share\_last\_2yrs &      -0.74 &  1.00 &  1.00 &       0.00 &  0.01 &  0.08 &    -0.29 &  0.02 &  0.10 &    -0.48 &  0.07 &  0.14 \\
foreigner\_

In [ ]:
table

program             job search                ... language              
metric                     OLS   perm   shap  ...      OLS   perm   shap
age                      0.004  0.053  0.152  ...   -0.020  0.143  0.305
female                   0.029  0.002  0.029  ...   -0.666  0.519  0.701
past_income             -0.084  0.766  0.676  ...    0.039  0.049  0.143
employability           -0.034  0.007  0.040  ...   -0.005  0.001  0.008
emp_share_last_2yrs     -0.737  1.000  1.000  ...   -0.478  0.071  0.139
foreigner_c              0.007  0.001  0.007  ...   -0.569  0.202  0.325
other_mother_tongue      0.021  0.002  0.031  ...   -0.942  1.000  1.000
unemp_rate               0.051  0.066  0.180  ...   -0.018  0.005  0.038
ue_cw_allocation4       -0.038  0.002  0.020  ...   -0.001  0.000  0.002

[9 rows x 12 columns]

In [ ]:
import statsmodels.api as sm

### Getting p-values:

In [ ]:
print(sm.OLS(cate["language"], X).fit().summary())

                            OLS Regression Results                            
Dep. Variable:               language   R-squared:                       0.966
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                 4.352e+04
Date:                Sun, 18 Apr 2021   Prob (F-statistic):               0.00
Time:                        00:38:33   Log-Likelihood:                 16998.
No. Observations:               62561   AIC:                        -3.391e+04
Df Residuals:                   62519   BIC:                        -3.353e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   3.5221    

## For DGP 0:

In [ ]:
cate = pd.read_csv("dataset_00.csv", index_col=0)
X = pd.read_csv("X.csv", index_col=0)

In [ ]:
estimators = ["R AML", "CF", "BCF", "BART"]

In [ ]:
index = pd.MultiIndex.from_product([estimators, ["OLS", "perm", "shap"]], names=["estimator", "metric"])

In [ ]:
results = pd.DataFrame(index=index, columns=X.columns)

In [ ]:
for estimator in estimators:
  estimates = cate[estimator]
  ols = sm.OLS(estimates, X).fit()
  results.loc[(estimator, "OLS")] = ols.params
  boost = xgboost.XGBRegressor(n_estimators=500).fit(X, estimates)
  perm = eli5.sklearn.PermutationImportance(boost, refit=False, random_state=0, n_iter=1)
  perm.fit(X, estimates)
  results.loc[(estimator, "perm")] = perm.feature_importances_
  explainer = shap.TreeExplainer(boost)
  shap_values = explainer(X)
  results.loc[(estimator, "shap")] = np.abs(shap_values.values).mean(axis=0)

[12:37:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:38:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:39:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:41:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
results.to_csv("DGP0 variable importances.csv")

In [ ]:
for estimator in estimators:
  for metric in ["perm", "shap"]:
    results.loc[(estimator, metric)] = results.loc[(estimator, metric)] / results.loc[(estimator, metric)].max()

In [ ]:
results.T

estimator                  R AML               ...         BART             
metric                       OLS         perm  ...         perm         shap
const                    1.03295            0  ...            0            0
age                    0.0559768    0.0185204  ...  0.000156537   0.00997315
canton_moth_tongue  -1.12432e-16            0  ...  3.42723e-05   0.00373999
city_big              0.00741101  0.000707766  ...  2.28907e-05   0.00230519
city_medium          -0.00296737  5.65629e-09  ...  2.19965e-05   0.00305531
city_no              -0.00422067            0  ...  2.45423e-06  0.000520287
cw_age               8.85793e-17  5.87802e-08  ...  1.72672e-05   0.00109382
cw_cooperative      -1.70708e-16            0  ...  7.77026e-06   0.00173297
cw_educ_above_voc    9.34582e-17  1.16625e-08  ...  0.000457205    0.0116907
cw_educ_tertiary    -5.91974e-17            0  ...  3.88847e-07  0.000201418
cw_female            9.02056e-17  1.57453e-09  ...    2.165e-06   0.00137183
cw_missing           2.10552e-16  6.03155e-10  ...  0.000207597   0.00633076
cw_own_ue           -9.69277e-17  2.84344e-08  ...  7.03711e-06   0.00248861
cw_tenure            1.98409e-17  8.72618e-09  ...  0.000170398   0.00112906
cw_voc_degree       -6.55942e-17  2.60336e-09  ...  0.000370292    0.0176074
emp_share_last_2yrs -8.97177e-17  6.40101e-08  ...  0.000563918    0.0229765
emp_spells_5yrs     -1.56376e-16  5.20198e-08  ...  0.000210687    0.0141979
employability        1.25009e-16            0  ...  4.20275e-05    0.0033858
female                  0.409393            1  ...            1            1
foreigner_b          7.06993e-11            0  ...  1.38925e-05   0.00191724
foreigner_c          8.75498e-11  2.84233e-10  ...            0            0
gdp_pc               1.33357e-17  5.73266e-08  ...  1.75787e-05   0.00200898
married              2.85796e-16   7.0838e-09  ...  0.000126513    0.0110347
other_mother_tongue  2.28767e-17            0  ...   3.3972e-05   0.00251306
past_income             0.360947     0.784792  ...      0.24474     0.402664
prev_job_manager      1.1254e-16  1.01039e-09  ...  0.000708261    0.0155268
prev_job_sec_mis      3.9452e-11  1.77097e-09  ...  0.000138566   0.00782586
prev_job_sec1        2.86438e-11  1.32007e-07  ...  1.55143e-05   0.00229071
prev_job_sec2         3.3286e-11            0  ...  1.41648e-06  0.000603931
prev_job_sec3         4.9749e-11  5.23579e-08  ...   8.4985e-06   0.00158169
prev_job_self        7.36173e-17  3.15055e-09  ...  2.09403e-05   0.00073784
prev_job_skilled     4.73796e-16  6.77958e-08  ...   5.9998e-07  0.000266744
prev_job_unskilled   2.01228e-16  2.71342e-07  ...  0.000410353    0.0192706
qual_semiskilled    -1.39119e-11  9.20292e-10  ...  9.43636e-05   0.00684178
qual_degree         -1.87682e-11  1.98825e-09  ...  9.65944e-08  0.000164279
qual_unskilled      -1.59726e-11  3.85457e-09  ...  1.12597e-07  0.000156849
qual_wo_degree      -6.68867e-12  1.08359e-09  ...  1.68952e-05   0.00148831
swiss                 9.9961e-11  8.11601e-08  ...  1.24054e-06   0.00077975
ue_cw_allocation1   -9.99634e-17  1.89572e-09  ...  2.61502e-05   0.00456507
ue_cw_allocation2   -5.13912e-17  7.29219e-10  ...  1.46987e-06   0.00101004
ue_cw_allocation3    1.22732e-16  4.74353e-08  ...  4.81071e-08  2.90896e-05
ue_cw_allocation4   -2.22912e-16  3.07932e-09  ...  5.78198e-08  1.47588e-05
ue_cw_allocation5   -6.88468e-18            0  ...  9.35604e-05   0.00677838
ue_cw_allocation6   -1.61546e-17            0  ...  7.41879e-06   0.00159015
ue_spells_last_2yrs -1.81875e-16  1.84472e-08  ...  4.01959e-06  0.000605138
unemp_rate          -1.13299e-17  9.22247e-08  ...  0.000115992    0.0091183

[46 rows x 12 columns]